# `ShouldProcess` for `-WhatIf`, `-Confirm` Support
We know and love the "try it out in a dry run" capability that PowerShell provides via the `-WhatIf` parameter :heart:. You have probably seen the `-Confirm` parameter on cmdlets, especially for ones that destroy (like `Remove-*` cmdlets).

We can support these same great features in our functions -- Thanks, Microsoft, for making it easy!

## The Gist

- What: find out if our invocation is likely to succeed, _without_ actually doing \<the thing>
- How: the `-WhatIf` parameter

and

- What: ensure that the caller really wants to do \<the thing>, _before_ actually doing \<the thing>, and without having reinvent wheels and writing user interface elements in our function
- How: the `-Confirm` parameter

## Examples of Confirmation Behavior

Some examples of how using the in-built PowerShell confirmation mechanisms enables user confirmation (and more!)

In [1]:
## prompted for confirmation by default
Write-Output jimmy, jams | .\SupportingShouldProcess\Remove-FictitiousThing.ps1 -Verbose

The below script needs to be able to find the current output cell; this is an easy method to get it.


Confirm
Are you sure you want to perform this action?
Performing the operation "Remove a fictitious thing" on target "Thingy 'jimmy'".
[Y] Yes  [A] Yes to All  [N] No  [L] No to All  [S] Suspend  [?] Help(default is 'Y')

Select:  y


VERBOSE: Bye bye, 'jimmy'

Confirm
Are you sure you want to perform this action?
Performing the operation "Remove a fictitious thing" on target "Thingy 'jams'".
[Y] Yes  [A] Yes to All  [N] No  [L] No to All  [S] Suspend  [?] Help(default is 'Y')

Select:  y


VERBOSE: Bye bye, 'jams'


In [2]:
## bonus: using ShouldProcess method in our code also enabled -WhatIf!
Write-Output jimmy, jams | .\SupportingShouldProcess\Remove-FictitiousThing.ps1 -WhatIf

What if: Performing the operation "Remove a fictitious thing" on target "Thingy 'jimmy'".
What if: Performing the operation "Remove a fictitious thing" on target "Thingy 'jams'".


In [3]:
## _not_ prompted for confirmation
Write-Output jimmy, jams | .\SupportingShouldProcess\Remove-FictitiousThing.ps1 -Verbose -Confirm:$false

VERBOSE: Performing the operation "Remove a fictitious thing" on target "Thingy 'jimmy'".
VERBOSE: Bye bye, 'jimmy'
VERBOSE: Performing the operation "Remove a fictitious thing" on target "Thingy 'jams'".
VERBOSE: Bye bye, 'jams'


## The Details
Microsoft enabled us to use the same framework and patterns for our functions that we are familiar with from many many cmdlets, and with minimal additional effort.

- What:  we can add, through familiar, established patterns, flexibility of our functions to support `-WhatIf` and `-Confirm` just like all the "real" cmdlets with which we (and our cohorts) are already familiar
- How: by making our function and "advanced" function (via `CmdletBinding`), and adding one conditional statement (for `ShouldProcess`)

To do so, we:
- add the `CmdletBinding` parameter attribute
- setting the `CmdletBinding` attribute's `SupportsShouldProcess` argument to `$true`
- setting the `CmdletBinding` attribute's `ConfirmImpact` argument to a value equal to- or higher than the default `$ConfirmPreference` preference automatic variable
- add a conditional statement based on the result of the method `$PSCmdlet.ShouldProcess()`
- place our "do this stuff" code into the scriptblock for the `$true` condition

See the sample code in the file [SupportingShouldProcess/Remove-FictitiousThing.ps1](./SupportingShouldProcess/Remove-FictitiousThing.ps1)

And, how this then works:
- PowerShell recognizes that the function does support ShouldProcess, thanks to the `SupportsShouldProcess` argument to the `CmdletBinding` attribute
- as the function code runs through to the `ShouldProcess()` method invocation, PowerShell assess the confirm impact and preferences
- when the `ConfirmImpact` setting on the function is equal to- or higher than the user's `$ConfirmPreference` preference automatic variable, PowerShell automatically engages the Confirm dialog
- the user gets the familiar, "Confirm    Are you sure you want to..."
- the Confirm message is controlled by the arguments to the `ShouldProcess()` method invocation
- bonus: this `ShouldProcess()` approach also enables `-WhatIf` support in the function -- yay!

## Further Reading
Some places from which to get additional information for these things:
- PowerShell help for [`about_Functions_CmdletBindingAttribute`](https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_functions_cmdletbindingattribute) -- the place that tells us about the `CmdletBinding` attribute, its arguments and uses, etc.
- [`ConfirmImpact` enumeration](https://docs.microsoft.com/en-us/dotnet/api/system.management.automation.confirmimpact) PowerShell SDK docs page -- so we know what are the available values to the `ConfirmImpact` argument to the `CmdletBinding` attribute
- PowerShell Deep Dive topic: [Everything you wanted to know about ShouldProcess](https://docs.microsoft.com/en-us/powershell/scripting/learn/deep-dives/everything-about-shouldprocess) -- more goodness on the topic